## Implementing a perceptron learning algorithm in Python

In [2]:
import numpy as np


class Perceptron(object):
    """Perceptron classifier.

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, shuffle=False, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.shuffle = shuffle
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            if self.shuffle:
                X, y = self._shuffle(X, y)
                
            for xi, target in zip(X, y):
                update = self.r * (target - self.predict(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self

    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = np.random.permutation(len(y))
        return X[r], y[r]

    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)

In [2]:
import struct
import numpy as np
from sklearn import svm, metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
class SVM(object):
    def __init__(self, C=0.1, eta=0.001, batch_size=1, max_iter=25, random_state=1):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.max_iter = max_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        
        ## initialize w
        
        m = np.shape(X)[0] # 행개수 60000
        n = np.shape(X)[1] # 열개수 784
        self.num_classes = len(np.unique(y)) # 클래스 수 = 10
        
        # y_encoded = self.encode_y(y)
        
        
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=X.shape[1])
        self.b_ = 0
        self.errors_ = []

        for _ in range(self.max_iter):
            errors = 0
            sum_w = np.zeros(X.shape[1])
            sum_b = 0.0
                
            for xi, target in zip(X, y):
                if(target * (np.dot(xi, self.w_) + self.b_)) < 1:
                    sum_w += (-target * xi)
                    sum_b += (-target)
            
            self.w_ = self.w_ - (self.eta * ((sum_w / X.shape[0]) + (1/self.C)*self.w_))
            self.b_ = self.b_ - self.eta * sum_b / X.shape[0]
        return self
    
    def _shuffle(self, X, y):
        """Shuffle training data"""
        r = np.random.permutation(len(y))
        return X[r], y[r]

    def predict(self, X):
        """Return class label after unit step"""
        predictions = []
        pred = np.dot(X, self.w_) + self.b_
        for val in pred:
            if(val >= 0):
                predictions.append(1)
            else:
                predictions.append(-1)
        return predictions

In [58]:
def read_idx(filename):
    with open(filename, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.fromstring(f.read(), dtype=np.uint8).reshape(shape)

In [59]:
## loading mnist dataset

raw_train = read_idx("./data/train-patterns-idx3-ubyte")
train_data = np.reshape(raw_train, (60000, 28*28))
train_label = read_idx("./data/train-labels-idx1-ubyte")

raw_test = read_idx("./data/test-patterns-idx3-ubyte")
test_data = np.reshape(raw_test, (10000, 28*28))
test_label = read_idx("./data/test-labels-idx1-ubyte")

c:\anaconda3\envs\dm2019\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  """


In [60]:
X = train_data
y = train_label

In [61]:
svm = SVM()

In [62]:
svm.fit(X,y)

In [63]:
y_pred = svm.predict(test_data)

In [67]:
test_label

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(test_data, y_pred)

ValueError: Classification metrics can't handle a mix of multiclass-multioutput and binary targets